In [0]:










%pip install --upgrade packaging

Python interpreter will be restarted.
  Attempting uninstall: packaging
    Found existing installation: packaging 21.2
    Not uninstalling packaging at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-754c325a-55a5-415d-94e1-cd5506bc9c9f
    Can't uninstall 'packaging'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 0.0.0 requires numpy==1.21.3, but you have numpy 1.24.3 which is incompatible.
notebook 0.0.0 requires packaging==21.2, but you have packaging 23.1 which is incompatible.
widgetsnbextension 3.6.1 requires notebook>=4.4.1, but you have notebook 0.0.0 which is incompatible.
Python interpreter will be restarted.


In [0]:
%pip list

Package                           Version
--------------------------------- --------------------
absl-py                           1.4.0
aiohttp                           3.8.5
aiosignal                         1.3.1
alembic                           1.11.3
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
asttokens                         2.0.5
astunparse                        1.6.3
async-timeout                     4.0.3
attrs                             21.2.0
azure-common                      1.1.28
azure-core                        1.29.2
azure-eventhub                    5.11.4
azure-identity                    1.14.0
azure-mgmt-core                   1.4.0
azure-mgmt-datafactory            3.1.0
azure-mgmt-subscription           3.1.1
backcall                          0.2.0
backports.entry-points-selectable 1.2.0
beautifulsoup4                    4.11.1
bertopic                          0.15.0
black                             22.3.0
bleach      

In [0]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import gensim

In [0]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"

In [0]:
# Reading PIMMART data
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

# Reading Syndigo 259K data
synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv') # 259k Syndigo Data
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
    synd_ALL[i] = synd_ALL[i].astype(np.float64)

# Trimming empty spaces from all columns
df_obj = synd_ALL.select_dtypes(['object'])
synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
syndigo_mapped = synd_ALL
pimmart = pim_gtin_mapped

In [0]:
syndigo_mapped.drop_duplicates('GTIN', inplace = True)

In [0]:
syndigo_mapped['ITEM_SUBCOM_text'] = \
(syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()

In [0]:
syndigo_mapped['Level 1'].value_counts()

Out[8]: Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances             

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range= (1,2), max_features = 800000)

syndigo_mapped['Level 1'].value_counts()

In [0]:
### import word2vec model trained on entire PIMMART data
from gensim.models import Word2Vec, KeyedVectors
model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model.most_similar('chips')

Out[9]: [('chips,', 0.7620069980621338),
 ('puffed/popped', 0.692810595035553),
 ('veggie/grain', 0.6568589806556702),
 ('chips.', 0.6461755037307739),
 ('popped', 0.6434525847434998),
 ('snacks', 0.6221342086791992),
 ('crisps', 0.6071634888648987),
 ('thins', 0.5962806344032288),
 ('<', 0.5704862475395203),
 ('kurokirishima', 0.5686668157577515)]

In [0]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

## RUN UNTIL HERE only ONCE

In [0]:
level__1 = "Beer / Wine / Spirits"
filenamee = "BEER_WINE_SPIRITS"

In [0]:
# level__1 = "Beer / Wine / Spirits"
# filenamee = "BEER_WINE_SPIRITS"

level__1 = "Cleaning & Janitorial"
filenamee = "CLEANING_JANITORIAL"

level__1 = "Toys / Games / Hobbies"
filenamee = "TOYS_GAMES_HOBBIES"

level__1 = "Office Supplies"

level__1 = "Building Supplies"
filenamee = "Building_Supplies"

level__1 = "Flooring"
filenamee = "FLOORING"

level__1 = "Beer / Wine / Spirits"
filenamee = "BEER_WINE_SPIRITS"


# Electronics

level__1 = "Food / Beverages"
filenamee = "Food / Beverages"
syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)

Out[12]: Meal Kits (Perishable)                                        309
Baby & Toddler Food                                          1623
Produce                                                      1649
Meat / Poultry / Seafood / Meat Substitutes (Perishable)     3976
Bakery / Deli                                                5333
Dairy & Egg Products                                         9018
Frozen Foods                                                13582
Beverages                                                   21240
Grocery                                                     74863
Name: Level 2, dtype: int64

#### word2vec Vectorization

In [0]:
level_1 = level__1
subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))
level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)

### LR Algorithm

In [0]:
lr_w2v = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga')
#print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
print(f"Training - w2v \"{level_1}\"")
warnings.filterwarnings("ignore", category=UserWarning)
lr_w2v.fit(X_train, y_train)

Training - w2v "Food / Beverages"
Out[18]: LogisticRegression(C=100, multi_class='multinomial', solver='saga')

In [0]:
print(' Display LR test  metrics')
preds_test = lr_w2v.predict(X_test)
preds_train = lr_w2v.predict(X_train)
print(classification_report(y_test, preds_test,labels = lr_w2v.classes_, target_names = [id2_level_map[i] for i in lr_w2v.classes_]))

 Display LR test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.89      0.88      0.88      1804
                                                 Grocery       0.91      0.96      0.93     14973
                                               Beverages       0.95      0.94      0.95      4248
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.69      0.68      0.68       795
                                            Frozen Foods       0.86      0.82      0.84      2716
                                           Bakery / Deli       0.58      0.30      0.39      1067
                                     Baby & Toddler Food       0.86      0.89      0.88       324
                                                 Produce       0.67      0.48      0.56       330
                                  Meal Kits (Perishable)       0.36      0.08      0.13    

### MLP Algorithm

In [0]:
mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
#print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
print(f"Training - w2v + MLP \"{level_1}\"")
warnings.filterwarnings("ignore", category=UserWarning)
mlp.fit(X_train, y_train)

Training - w2v + MLP "Food / Beverages"
Out[20]: MLPClassifier(early_stopping=True, hidden_layer_sizes=(200,),
              learning_rate='adaptive')

In [0]:
print(' Display MLP test  metrics')
preds_test_mlp = mlp.predict(X_test_w2v)
preds_train_mlp = mlp.predict(X_train_w2v)
print(classification_report(y_test_w2v, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))

 Display MLP test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.92      0.92      0.92      1804
                                                 Grocery       0.94      0.96      0.95     14973
                                               Beverages       0.97      0.95      0.96      4248
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.73      0.66      0.70       795
                                            Frozen Foods       0.87      0.90      0.88      2716
                                           Bakery / Deli       0.61      0.46      0.52      1067
                                     Baby & Toddler Food       0.86      0.93      0.90       324
                                                 Produce       0.64      0.63      0.63       330
                                  Meal Kits (Perishable)       0.35      0.13      0.19   

### Oversampling minority classes

In [0]:
%pip install -U imbalanced-learn

Python interpreter will be restarted.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Not uninstalling numpy at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-277910b2-e14e-482f-b007-5383ddcab73d
    Can't uninstall 'numpy'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 0.0.0 requires numpy==1.21.3, but you have numpy 1.22.4 which is incompatible.
widgetsnbextension 3.6.1 requires notebook>=4.4.1, but you have notebook 0.0.0 which is incompatible.
statsmodels 0.13.2 requires packaging>=21.3, but you have packaging 21.2 which is incompatible.
Python interpreter will be restarted.


In [0]:
#Original Level Counts
dict(pd.Series(y_train).value_counts())

Out[30]: {1: 59890,
 2: 16992,
 4: 10866,
 0: 7214,
 5: 4266,
 3: 3181,
 7: 1319,
 6: 1299,
 8: 247}

In [0]:
desired_levelcount = {1: 59890,
 2: 16992,
 4: 10866,
 0: 10000,
 5: 10000,
 3: 10000,
 7: 10000,
 6: 10000,
 8: 10000}

In [0]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= desired_levelcount ).fit_resample(X_train, y_train)
print('Before Resampling:\n',pd.Series(y_train).value_counts())
print('After Resampling:\n',pd.Series(y_resampled).value_counts())

Before Resampling:
 1    59890
2    16992
4    10866
0     7214
5     4266
3     3181
7     1319
6     1299
8      247
dtype: int64
After Resampling:
 1    59890
2    16992
4    10866
0    10000
3    10000
7    10000
5    10000
6    10000
8    10000
dtype: int64


In [0]:
mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
#print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
print(f"Training - w2v + MLP \"{level_1}\"")
warnings.filterwarnings("ignore", category=UserWarning)
mlp_smote.fit(X_resampled, y_resampled)

Training - w2v + MLP "Food / Beverages"
Out[34]: MLPClassifier(early_stopping=True, hidden_layer_sizes=(200,),
              learning_rate='adaptive')

In [0]:
print(' Display MLP + SMOTE test  metrics')
preds_test_mlp_smote = mlp_smote.predict(X_test)
preds_train_mlp_smote = mlp_smote.predict(X_resampled)
print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))

 Display MLP + SMOTE test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.91      0.91      0.91      1804
                                                 Grocery       0.94      0.94      0.94     14973
                                               Beverages       0.95      0.95      0.95      4248
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.66      0.73      0.70       795
                                            Frozen Foods       0.86      0.88      0.87      2716
                                           Bakery / Deli       0.56      0.47      0.51      1067
                                     Baby & Toddler Food       0.84      0.92      0.88       324
                                                 Produce       0.57      0.62      0.60       330
                                  Meal Kits (Perishable)       0.31      0.27     

#### IGNORE BELOW

preds = lr_tf.predict(X_test)
preds_lrtf = preds
probs = lr_tf.predict_proba(X_test)
preds_train = lr_tf.predict(X_train)
probs_train = lr_tf.predict_proba(X_train)
print(f"Done Training - \"{level_1}\"")

level_num = []
testProobs = []
number = 1

print(len(preds))
for i in range(len(preds)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])

test_new_proobs = []
for i in range(len(testProobs)):
    test_new_proobs.append(testProobs[i].tolist())
test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]

level_num = []
trainProobs = []

print(len(preds_train))
for i in range(len(preds_train)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    trainProobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1]) 

train_new_proobs = []
for i in range(len(trainProobs)):
    train_new_proobs.append(trainProobs[i].tolist())
train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]

testLevels = []
for j in y_test:
    testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])

testLevelss = []
for j in preds:
    testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

trainLevelss = []
for j in preds_train:
    trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

print("---------------------------------------------\nFINAL COUNTS:\n---------------------------------------------")
print("1)", len(A_test + A_train))
print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
print("3)", len(B_test + B_train))
print("4)", len(testLevels + B_train))
print("5)", len(testLevelss + trainLevelss))
print("6)", int(len(test_new_proobs + train_new_proobs)))

data = {
    'GTIN' : A_test + A_train,
    'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
    'Actual Level 2' : B_test + B_train,
    'Actuals' : testLevels + B_train,
    'Predictions' : testLevelss + trainLevelss,
    'Scores' : test_new_proobs + train_new_proobs,
    'L1 Name' : level_1
}

df = pd.DataFrame(data)
df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
    'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
    'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
    'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
    'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]

## Keep the output of the below cell for all Level 1 executed

df.to_csv(DBFO + filenamee + '.csv', index=False)
print(f"{filenamee}.csv")